In [1]:
import os
import xarray as xr
import numpy as np
from glob import glob
import pandas as pd
import trackpy as tp
import proplot as pplt
from copy import deepcopy
from pyresample import kd_tree
from pyresample.geometry import SwathDefinition, GridDefinition

In [2]:
!ls ../data/clean_lightning/flash/

S5P_PAL__L2__NO2____20190810T212136_20190810T230306_09456_01_020301.nc
S5P_PAL__L2__NO2____20190810T230306_20190811T004435_09457_01_020301.nc
S5P_PAL__L2__NO2____20190811T004435_20190811T022605_09458_01_020301.nc
S5P_PAL__L2__NO2____20190811T022605_20190811T040734_09459_01_020301.nc
S5P_PAL__L2__NO2____20190811T040734_20190811T054904_09460_01_020301.nc


In [3]:
grid_lon = np.arange(-180, 180+0.125, 0.125)
grid_lat = np.arange(60, 90+0.125, 0.125)
lons, lats = np.meshgrid(grid_lon, grid_lat)
arctic_grid = GridDefinition(lons=lons, lats=lats)

In [4]:
def segmentation(frame, filename):
    '''Get the features of TROPOMI swath segmentation'''
    ds = xr.open_dataset(filename, group='S5P')

    # swath_tropomi = SwathDefinition(lons=ds['longitude'], lats=ds['latitude'])
    # da_resample = xr.DataArray(kd_tree.resample_nearest(swath_tropomi, ds['lightning_mask'].values,
    #                                                     arctic_grid, radius_of_influence=50000, epsilon=0.5),
    #                         dims=['latitude', 'longitude'], coords=[grid_lat, grid_lon])

    labels = np.unique(ds['lightning_mask'])
    labels = np.delete(labels, 0)

    lon_centers = []; lat_centers = []

    for label in labels:
        mask = ds['lightning_mask']==label
        mask_region = ds['lightning_mask'].where(mask, drop=True)
        lon_centers.append(mask_region['longitude'].mean())
        lat_centers.append(mask_region['latitude'].mean())

    # define the swath based on lightning mask centers
    swath_center = SwathDefinition(lons=lon_centers, lats=lat_centers)

    # Determine nearest (w.r.t. great circle distance) neighbour in the grid.
    _, _, index_array, distance_array = kd_tree.get_neighbour_info(source_geo_def=arctic_grid,
                                                                   target_geo_def=swath_center,
                                                                   radius_of_influence=80000,
                                                                   neighbours=1)

    # get the y and x index
    y, x = np.unravel_index(index_array, arctic_grid.shape)

    feature = xr.Dataset({'frame': (['index'], [frame]*len(y)),
                          'hdim_1': (['index'], y),
                          'hdim_2': (['index'], x),
                          'longitude': (['index'], lon_centers),
                          'latitude': (['index'], lat_centers),
                          'label': (['index'], labels),
                          'filename': (['index'], [os.path.basename(filename)]*len(y))
                          })

    return feature

In [5]:
features = []

for frame,filename in enumerate(sorted(glob('../data/clean_lightning/flash/S5P_PAL__L2_*'))):
    features.append(segmentation(frame, filename))

features = xr.concat(features, dim='index')

In [6]:
# keyword arguments for linking step
parameters_linking={}
parameters_linking['dxy']=5000 # m
parameters_linking['dt']=1.5*3600 # s
parameters_linking['vmax']=500 # m/s, actually this doesn't stand for the vmax because the NO2 mask is larger than the lightning/cloud region
parameters_linking['stubs']=2
parameters_linking['order']=1
parameters_linking['extrapolate']=1
parameters_linking['memory']=0
parameters_linking['adaptive_stop']=0.2
parameters_linking['adaptive_step']=0.95
parameters_linking['subnetwork_size']=100
parameters_linking['method_linking']= 'predict'

In [7]:
pred = tp.predict.NearestVelocityPredict(span=1)
tp.linking.Linker.MAX_SUB_NET_SIZE = parameters_linking['subnetwork_size']

trajectories = pred.link_df(deepcopy(features.to_dataframe()),
    search_range=int(parameters_linking['dt'] * parameters_linking['vmax'] / parameters_linking['dxy']),
    memory=parameters_linking['memory'],
    pos_columns=["hdim_1", "hdim_2"],
    t_column="frame",
    neighbor_strategy="KDTree",
    link_strategy="auto",
    adaptive_step=parameters_linking['adaptive_step'],
    adaptive_stop=parameters_linking['adaptive_stop']
)

# assigning incremental values based on an unique value of a column
trajectories["cell"] = pd.factorize(trajectories["particle"])[0] + 1
trajectories.drop(columns=["particle"], inplace=True)

# remove segmentations which only have one time step linking
trajectories = trajectories.groupby('cell').filter(lambda x : len(x)>1).to_xarray()

Frame 4: 1 trajectories present.


In [8]:
trajectories

<xarray.Dataset>
Dimensions:    (index: 4)
Coordinates:
  * index      (index) int64 0 1 2 3
Data variables:
    frame      (index) int64 0 1 2 3
    hdim_1     (index) int64 199 200 203 204
    hdim_2     (index) int64 2358 2354 2367 2449
    longitude  (index) float32 114.8 114.3 115.9 126.1
    latitude   (index) float32 84.84 84.94 85.37 85.5
    label      (index) int64 1 1 1 1
    filename   (index) object 'S5P_PAL__L2__NO2____20190810T212136_20190810T2...
    cell       (index) int64 1 1 1 1